In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize
import numpy as np
import seaborn as sns
from google.colab import drive

# Setup Environment and Import Libraries
drive.mount('/content/drive')

# Define Data Paths
train_dir = '/content/drive/My Drive/Dataset/Dataset Fish Species/Types of fish/FishImgDataset/train'
validation_dir = '/content/drive/My Drive/Dataset/Dataset Fish Species/Types of fish/FishImgDataset/val'
new_data_dir = '/content/drive/My Drive/Dataset/Dataset Fish Species/Types of fish/FishImgDataset/test'

# Set target image size
target_size = (150, 150)

# Prepare Image Data Generators (Preprocessing Data)
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    new_data_dir,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Build the CNN Model with input shape matching the new image size
model = Sequential([
    Conv2D(32, (3, 3), input_shape=(150, 150, 3)),
    LeakyReLU(alpha=0.1),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3)),
    LeakyReLU(alpha=0.1),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128),
    LeakyReLU(alpha=0.1),

    Dense(train_generator.num_classes, activation='softmax')  # Use softmax for multi-class classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define Early Stopping Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the Model
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

# Evaluate the Model on Test Data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Save the model
model_path = '/content/drive/My Drive/classification_of_fish_species.h5'
model.save(model_path)


Mounted at /content/drive
Found 8801 images belonging to 31 classes.
Found 2751 images belonging to 31 classes.
Found 1760 images belonging to 31 classes.
Epoch 1/15
276/276 [==============================] - 3543s 13s/step - loss: 2.7354 - accuracy: 0.2866 - val_loss: 1.9628 - val_accuracy: 0.4846
Epoch 2/15
276/276 [==============================] - 93s 338ms/step - loss: 1.5495 - accuracy: 0.5750 - val_loss: 0.8223 - val_accuracy: 0.7957
Epoch 3/15
276/276 [==============================] - 92s 334ms/step - loss: 0.6106 - accuracy: 0.8316 - val_loss: 0.3961 - val_accuracy: 0.9095
Epoch 4/15
276/276 [==============================] - 93s 336ms/step - loss: 0.1694 - accuracy: 0.9577 - val_loss: 0.2930 - val_accuracy: 0.9429
Epoch 5/15
276/276 [==============================] - 94s 341ms/step - loss: 0.0482 - accuracy: 0.9901 - val_loss: 0.3035 - val_accuracy: 0.9527
Epoch 6/15
276/276 [==============================] - 94s 339ms/step - loss: 0.0380 - accuracy: 0.9925 - val_loss: 0.306

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
!pip install tensorflowjs

# Buat direktori output jika belum ada
import os
output_dir = '/content/drive/MyDrive/tfjs_model'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Convert Keras model to TensorFlow.js format
!tensorflowjs_converter --input_format keras '/content/drive/MyDrive/classification_of_fish_species.h5' '/content/drive/MyDrive/tfjs_model'


2024-06-14 05:37:26.301335: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-14 05:37:26.305168: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-14 05:37:26.358285: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 05:37:27.339524: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
